# Outliers and Missing Data (EDA)

## 1. 실험 환경 변수

### 사용 패키지

In [ ]:
import os
from datetime import datetime
from glob import glob
from typing import Dict, List, Tuple

import geopandas as gpd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from metr.components import Metadata, TrafficData

plt.rcParams["font.family"] = "AppleGothic"
plt.rcParams["axes.unicode_minus"] = False  # 음수 부호 깨짐 방지

### 주요 데이터 경로

In [ ]:
MAP_DATA_OF_SENSORS = "../datasets/metr-imc/nodelink/imc_link.shp"
TRAFFIC_RAW_PATH = "../datasets/metr-imc/metr-imc.h5"
METADATA_RAW_PATH = "../datasets/metr-imc/metadata.h5"
OUTLIER_OUTPUT_DIR = "./output/outlier_processed"
INTERPOLATED_OUTPUT_DIR = "./output/interpolated"
FINAL_OUTPUT_DIR = "./output/final"
PREDICTION_OUTPUT_DIR = "./output/prediction"

os.makedirs(OUTLIER_OUTPUT_DIR, exist_ok=True)
os.makedirs(INTERPOLATED_OUTPUT_DIR, exist_ok=True)
os.makedirs(FINAL_OUTPUT_DIR, exist_ok=True)
os.makedirs(PREDICTION_OUTPUT_DIR, exist_ok=True)

## 2. Raw Data

### Raw 데이터 로딩

Raw 데이터는 원래 인천시에서 제공되었던 형태에서 현재 METR-LA와 같은 형태로 변환되어 있다. 행은 시간을 나타내고 열은 각 도로의 센서를 나타낸다. 형식은 전국표준노드링크의 Link ID를 나타내며 string 형식으로 지정되어 있다.

이 연구에서 사용되는 모든 데이터는 아래와 같은 형태를 사용한다. 이것은 METR-LA 데이터셋과 유사한 구조이다.

In [ ]:
raw = TrafficData.import_from_hdf(TRAFFIC_RAW_PATH)
raw_data = raw.data
raw_data

원래 표준노드링크는 전국의 데이터를 포함하며 아래는 그 중 인천시에 해당하는 데이터만 추출한 데이터이다. 해당 데이터는 2024년 3월 25일에 최신화된 데이터이다.

In [ ]:
geo_gdf = gpd.read_file(MAP_DATA_OF_SENSORS)
print(geo_gdf.shape)
geo_gdf.explore()

하지만, 인천시에서는 모든 도로에 대해 교통량을 측정하지 않으며 교통량을 측정하고 있는 도로는 다음과 같다

In [ ]:
# geo_gdf에서 LINK_ID가 raw_df.columns에 있는 것만 남기기
geo_gdf_with_traffic: gpd.GeoDataFrame = geo_gdf[geo_gdf["LINK_ID"].isin(raw_data.columns)]

print(geo_gdf_with_traffic.shape)
geo_gdf_with_traffic.explore()

메타데이터는 표준노드링크에 명시된 내용을 추출했으며 각 센서에 해당하는 도로의 정보를 포함한다. 본 연구에서는 도로 허용 용량을 계산할 때에만 사용한다.

In [ ]:
raw_metadata_df = Metadata.import_from_hdf(METADATA_RAW_PATH).data
raw_metadata_df

### 기본적인 결측치 처리

결측치 비율이 전체 데이터의 50% 이상인 데이터를 선정하여 보간이 제대로 이루어 질 수 있을 정도의 데이터를 추출

In [ ]:
# 데이터 총 개수
total_raw_length = raw_data.shape[0]
total_raw_length

# 50% 이상의 결측값을 가진 센서 제거
filtered_raw_df = raw_data.dropna(thresh=int(total_raw_length * 0.5), axis=1)
filtered_raw_df

In [ ]:
# 제거된 센서 개수
print("Removed:", raw_data.shape[1] - filtered_raw_df.shape[1])
print("Ratio:", (raw_data.shape[1] - filtered_raw_df.shape[1]) / raw_data.shape[1] * 100, "%")
removed_sensors = raw_data.columns.difference(filtered_raw_df.columns)
removed_sensors

### Description of Raw Data

In [ ]:
print("Shape of the dataset:", filtered_raw_df.shape)
print("Description of the Incheon dataset")
print(filtered_raw_df.reset_index(drop=True).melt()["value"].describe().apply(lambda x: format(x, '.4f')))
print()

total_missing_values_count = filtered_raw_df.isnull().sum().sum()
print("Total missing values count:", total_missing_values_count, f"({total_missing_values_count / filtered_raw_df.size * 100:.4f}%)")
print()

sensors_with_mv = filtered_raw_df.isnull().any(axis=0)
print("Sensors with missing values count:", sensors_with_mv.sum(), f"({sensors_with_mv.sum() / filtered_raw_df.shape[1] * 100:.4f}%)")
print("Sensors with no missing values count:", (~sensors_with_mv).sum(), f"({(~sensors_with_mv).sum() / filtered_raw_df.shape[1] * 100:.4f}%)")

### Plots of Raw Data

In [ ]:
def plot_mean_by_time(df: pd.DataFrame, title: str, size: Tuple=(10, 6)):
    groups = df.groupby([df.index.hour, df.index.minute, df.index.second])
    mean_df = groups.apply(lambda x: x.stack().mean())
    mean_df.index = [
        datetime(year=1970, month=1, day=1, hour=h, minute=m, second=s)
        for h, m, s in mean_df.index
    ]
    
    plt.figure(figsize=size)
    sns.lineplot(x=mean_df.index, y=mean_df.values)
    
    ax = plt.gca()
    # x축을 00:00:00부터 23:00:00까지 설정
    ax.set_xlim([datetime(1970, 1, 1, 0, 0, 0), datetime(1970, 1, 1, 23, 0, 0)])
    # 1시간 간격으로 눈금 표시
    ax.xaxis.set_major_locator(mdates.HourLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H"))
    
    plt.xlabel("Time")
    plt.ylabel("Average Value")
    plt.title(title)
    plt.grid(True)
    plt.show()


def plot_by_time(df: pd.DataFrame, title: str, size: Tuple=(12, 6)):
    plt.figure(figsize=size)
    sns.lineplot(x=df.index, y=df.values)
    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

    plt.xlabel("Time")
    plt.ylabel("Average Value")
    plt.title(title)

    plt.grid(True)

    plt.show()


def plot_by_days(df: pd.DataFrame, name: str):
    data_list = []

    for day in range(7):
        day_data = df[df.index.dayofweek == day]
        numeric_cols = day_data.select_dtypes(include=[np.number]).columns
        day_numeric = day_data[numeric_cols]
        values = day_numeric.values.flatten()
        values = values[~np.isnan(values)]
        df_day = pd.DataFrame({"Volumes": values, "Days": day})
        data_list.append(df_day)

    df_all = pd.concat(data_list, ignore_index=True)

    # 요일 레이블 매핑
    day_labels = {0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"}
    df_all["Days"] = df_all["Days"].map(day_labels)
    df_all["Days"] = pd.Categorical(
        df_all["Days"],
        categories=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
        ordered=True,
    )

    plt.figure(figsize=(10, 6))
    sns.boxplot(
        x="Days",
        y="Volumes",
        data=df_all,
        order=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
    )
    plt.xlabel("Days")
    plt.ylabel("Traffic Volumes")
    plt.title(f"Data by Day in {name}")
    plt.grid(True)

    ax = plt.gca()
    formatter = FuncFormatter(lambda x, pos: f"{int(x):,}")
    ax.yaxis.set_major_formatter(formatter)

    plt.show()


def plot_hist(
    df: pd.DataFrame,
    name: str,
    exclude_zero: bool = False,
    y_max: float = None,
    x_max: float = None,
    bins: int = 50,
):
    values = df.values.flatten()
    values = values[~np.isnan(values)]

    if exclude_zero:
        values = values[values != 0]

    if x_max is not None:
        bin_edges = np.histogram_bin_edges(
            values, bins=bins, range=(values.min(), x_max)
        )
        bin_edges = np.append(bin_edges, np.inf)
    else:
        bin_edges = bins

    plt.figure(figsize=(10, 6))
    sns.histplot(values, bins=bin_edges, kde=True)
    plt.xlabel("Value")
    plt.title(f"Histogram of {name}")
    plt.grid(True)

    if y_max is not None:
        plt.ylim(top=y_max)

    ax = plt.gca()
    formatter = FuncFormatter(lambda x, _: f"{int(x):,}")
    ax.yaxis.set_major_formatter(formatter)
    ax.xaxis.set_major_formatter(formatter)

    plt.show()

아래 그래프는 전체 데이터를 평균을 내고 추세를 본 것이다. 일반적으로 대부분의 각 센서의 데이터는 아래와 같은 형태를 보인다.

In [ ]:
plot_mean_by_time(filtered_raw_df, "Plots by Time")

아래는 요일 별 데이터의 분포를 나타낸 것이다. 2024년 3월 이전의 데이터는 말도 안 되는 값(>100000)들이 많이 있는 것을 확인하기도 했지만 현재 범위(2024-03 ~ 2024-09)의 데이터는 극단적인 값들은 크게 없는 것으로 보인다.

In [ ]:
plot_by_days(filtered_raw_df, "Plots by Days")

아래는 데이터 구간 별 데이터 분포를 나타낸다. 20000만개 이상은 그래프에 표시하지 않았지만 대부분의 데이터는 교통량이 2000아래에 분포해 있다. 또한 교통량이 2000이상 넘어가는 도로는 대체로 고속도로이다. 다만, 이상치도 포함하고 있다.

In [ ]:
plot_hist(filtered_raw_df, "Data Histogram", y_max=20000)

일반적으로 데이터의 그래프는 아래와 같다. 특정 기간에서 데이터가 증가한 것을 확인할 수 있다. 그 외에도 어떤 그래프의 경우 말도 안되는 값을 가지는 경우도 확인할 수 있다.

In [ ]:
target_data = filtered_raw_df.iloc[:, 1]
plot_by_time(target_data, f"Plots by Time: {target_data.name}")
geo_gdf[geo_gdf["LINK_ID"] == target_data.name].explore(
    style_kwds={
        "color": "red",  # 선 색상
        "weight": 5,  # 선 굵기 (픽셀 단위)
    },
)

In [ ]:
# 고속도로
target_data = filtered_raw_df.loc[:, "1610002900"]
plot_by_time(target_data, f"Plots by Time: {target_data.name}")
geo_gdf[geo_gdf["LINK_ID"] == target_data.name].explore(
    style_kwds={
        "color": "red",  # 선 색상
        "weight": 5,  # 선 굵기 (픽셀 단위)
    },
)

2024년 3월 이전의 데이터는 100,000이 넘어가는 비정상적 데이터도 확인했지만 현재 범위의 데이터에서는 비정상적으로는 보이지 않는다

In [ ]:
idx = -1

In [ ]:
while idx < filtered_raw_df.shape[1]:    
    idx += 1
    target_data = filtered_raw_df.iloc[:, idx]
    if target_data[target_data > 10000].any():
        break

plot_by_time(target_data, f"Plots by Time: {target_data.name}")
geo_gdf[geo_gdf["LINK_ID"] == target_data.name].explore(
    style_kwds={
        "color": "red",  # 선 색상
        "weight": 5,  # 선 굵기 (픽셀 단위)
    },
)
        

"1640332801"과 같은 센서의 경우 결측치가 대부분이고 유효한 값은 거의 없는 것으로 보인다. 해당 위치의 경우 차량통행이 꽤 있는 곳이며 측정이 제대로 이루어지지 않았음을 예측할 수 있다.

In [ ]:
idx = -1

In [ ]:
while idx < filtered_raw_df.shape[1]:    
    idx += 1
    target_data = filtered_raw_df.iloc[:, idx]
    if target_data.isna().sum() > 1000:
        break

plot_by_time(target_data, f"Plots by Time: {target_data.name}")
geo_gdf[geo_gdf["LINK_ID"] == target_data.name].explore(
    style_kwds={
        "color": "red",  # 선 색상
        "weight": 5,  # 선 굵기 (픽셀 단위)
    },
)

## 3. Outliers 처리

### 기본 Outlier 처리

앞서 본 연구에서 다음 이상치 처리를 기본으로 적용한다.

1. 결측치 주변 0을 제거
  경험적 데이터 분석 결과를 바탕으로 결측치 주변 0이 
2. 이론적 도로 허용 용량 기반 제거 (1.5배 또는 2.0배까지 허용)

In [ ]:
from songdo_rnn.preprocessing.outlier import remove_base_outliers

In [ ]:
result = remove_base_outliers(
    filtered_raw_df,
    start_datetime=None,
    end_datetime=None,
    traffic_capacity_adjustment_rate=2.0,
    metadata_path=METADATA_RAW_PATH,
)
result_path = os.path.join(OUTLIER_OUTPUT_DIR, "base.h5")
result.to_hdf(result_path, key="data")

In [ ]:
base_df = TrafficData.import_from_hdf(result_path).data
base_df

### 테스트 가능한 데이터만 추출

In [ ]:
test_start = "2024-08-01"
test_end = "2024-08-31"
test_candidates = filtered_raw_df.loc[test_start:test_end, :]
test_candidates

In [ ]:
na_exist_list = test_candidates.columns[test_candidates.isna().any()]
intersection_list = list(base_df.columns.intersection(na_exist_list))
print(len(na_exist_list))
print("NaNs:", len(intersection_list), "/", len(base_df.columns))

In [ ]:
non_intersection_columns = base_df.columns.difference(intersection_list)
training_df = base_df[non_intersection_columns]
training_df = training_df.loc[:pd.Timestamp(test_start), :]
training_df = training_df.iloc[:-1, :]
training_df

In [ ]:
test_df = filtered_raw_df.loc[test_start:test_end, training_df.columns]
test_df

### 기본 Outlier 처리 결과 시각화

In [ ]:
def compare_fromto_df(df1: pd.Series, df2: pd.Series, title: str = ""):
    plt.figure(figsize=(12, 6))
    df1.plot(label="Original Data", color="red")
    df2.plot(label="Processed Data", color="blue")
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
idx = -1

In [ ]:
idx += 1

target_data = training_df.iloc[:, idx]
original_data = raw_data[target_data.name]


compare_fromto_df(original_data, target_data, title=f"Origin vs Target: {original_data.name}")

In [ ]:
idx = -1

In [ ]:
while idx < training_df.shape[1]:
    idx += 1
    target_data = training_df.iloc[:, idx]
    original_data = filtered_raw_df[target_data.name]
    original_data = original_data.loc[target_data.index.min():target_data.index.max()]
    

    if original_data.isna().sum() == target_data.isna().sum():
        continue

    print(original_data.name, ":" , original_data.isna().sum())
    print(target_data.name, ":", target_data.isna().sum())

    compare_fromto_df(original_data, target_data, title=f"Origin vs Target: {original_data.name}")

    break

In [ ]:
start_idx = 0
end_idx = 0

for idx in range(target_data.shape[0]):
    if pd.isna(target_data.iloc[idx]):
        start_idx = idx - 10
        break


for idx in range(target_data.shape[0] - 1, -1, -1):
    if pd.isna(target_data.iloc[idx]):
        end_idx = idx + 10
        break

start_idx = start_idx if start_idx > 0 else 0
end_idx = end_idx if end_idx < target_data.shape[0] else target_data.shape[0]

original_cut_data = original_data.iloc[start_idx:end_idx]
target_cut_data = target_data.iloc[start_idx:end_idx]

compare_fromto_df(original_cut_data, target_cut_data, title=f"Origin vs Target: {original_data.name}")



아래 코드는 0제거 외에 허용 용량에 따른 제거된 사례를 확인하는 코드임. 테스트 상 2.0배를 넘어가는 경우는 종종 있지만 2.5를 넘어가지는 않음. 대체로 속도가 바뀌는 지점의 도로에서 과속이 많이 발생하고 이에 따라 허용 용량 이상으로 차량이 통과하는 것으로 보임. 따라서, 실제 통과 차량의 한계를 얼마나 정할 것인지 정할 필요가 있음.

In [ ]:
idx = -1

In [ ]:
while idx < training_df.shape[1] - 1:
    idx += 1
    target_data = training_df.iloc[:, idx]
    original_data = filtered_raw_df[target_data.name]
    original_data = original_data.loc[target_data.index.min():target_data.index.max()]
    

    if original_data.isna().sum() == target_data.isna().sum():
        continue
    
    # 0이 아닌 값 중 NaN이 아닌 값만 추출
    # 즉 실제 삭제된 결측치만 추출
    not_0_and_na = original_data[original_data != 0]
    not_0_and_na = not_0_and_na[not_0_and_na.notna()]
    na_target = target_data[not_0_and_na.index]
    if na_target.isna().sum() == 0:
        continue

    print(original_data.name, "(Origin):" , original_data.isna().sum())
    print(target_data.name, "(Target):", target_data.isna().sum())
    metadata = raw_metadata_df[raw_metadata_df["LINK_ID"] == original_data.name]
    print(metadata)

    # Capacity 계산 과정
    speed_limit = metadata["MAX_SPD"].values[0]
    alpha = 10 * (100 - speed_limit)
    if speed_limit > 100:
        alpha /= 2
    lane_count = metadata["LANES"].values[0]
    adjustment_rate = 2.0
    original_capacity = (2200 - alpha) * lane_count
    capacity = original_capacity * adjustment_rate
    print("Capacity:", capacity, f"({original_capacity} x {adjustment_rate})")
    exceed_capacity_data = target_data[target_data > capacity]
    print(exceed_capacity_data if exceed_capacity_data.size > 0 else "No exceed capacity data")

    compare_fromto_df(original_data, target_data, title=f"Origin vs Target: {original_data.name}")
    
    break

print(original_data.name)
geo_gdf[geo_gdf["LINK_ID"] == original_data.name].explore()

### 테스트 데이터 임의 결측치 생성

rnn_outliers에서 좀 더 개선되고 변경된 코드로 동작

In [ ]:
# 테스트 데이터에 NaN이 없는지 다시 확인
test_df.isna().sum().sum()

In [ ]:
# 이상치 데이터의 범위
t_all = training_df.values.flatten()
t_all = t_all[~np.isnan(t_all)]
t_mean = t_all.mean()
t_std = t_all.std()
t_z = (t_all - t_mean) / t_std
t_outlier_indices = np.where(np.abs(t_z) > 3)[0]
t_outliers = t_all[t_outlier_indices]

# 이상치 데이터의 범위와 통계 출력
if len(t_outliers) > 0:
    min_outlier = t_outliers.min()
    max_outlier = t_outliers.max()
    print(f"이상치 데이터의 개수: {len(t_outliers)}")
    print(f"이상치 데이터의 범위: {min_outlier:.2f}에서 {max_outlier:.2f}")
    print(f"전체 데이터의 평균: {t_mean:.2f}, 표준편차: {t_std:.2f}")
    print(
        f"전체 데이터의 정상 범위: {t_mean - 3 * t_std:.2f}에서 {t_mean + 3 * t_std:.2f}"
    )
    print(f"이상치 비율: {len(t_outliers) / len(t_all) * 100:.2f}%")
else:
    print("z-score가 3 이상인 이상치 데이터가 없습니다.")

In [ ]:
# 전체 결측치 비율
missing_ratio = training_df.isna().sum().sum() / training_df.size
print(f"전체 결측치 비율: {missing_ratio:.4f} ({missing_ratio*100:.2f}%)")

# 열별 결측치 비율
column_missing = training_df.isna().mean()
print("\n열별 결측치 비율:")
print(column_missing)

# 결측치가 가장 많은 열 확인
print("\n결측치가 가장 많은 열 Top 5:")
print(column_missing.sort_values(ascending=False).head())

# 행별 결측치 비율 분포 확인
row_missing = training_df.isna().mean(axis=1)
print(f"\n행별 결측치 비율 평균: {row_missing.mean():.4f}")
print(f"행별 결측치 비율 최댓값: {row_missing.max():.4f}")

In [ ]:
t_outliers.sort()
t_outliers

In [ ]:
# test_df의 copy 생성
# 해당 copy에 이상치를 위에서 구한 이상치 비율만큼 데이터를 랜덤 선택하고 t_outliers의 랜덤을 선택한 어느 한 값으로 대체
# 위 데이터에 위에서 구한 전체 결측치 비율만큼 데이터를 랜덤 선택하고 NaN 결측치로 대체

# test_df의 copy 생성
corrupted_test_df = test_df.copy()

# 1. 이상치 비율 계산 (이전 셀에서 이미 계산됨)
outlier_ratio = len(t_outliers) / len(t_all)

# 2. 결측치 비율 계산
missing_ratio = training_df.isna().sum().sum() / training_df.size

# 3. 이상치 추가 - 이상치 비율만큼 랜덤 데이터 선택하고 t_outliers 값으로 대체
# 전체 데이터 포인트 개수
total_points = corrupted_test_df.size
# 추가할 이상치 개수
n_outliers = int(total_points * outlier_ratio)

# 랜덤으로 위치 선택 (행, 열 인덱스)
random_indices = np.random.choice(total_points, n_outliers, replace=False)
rows = random_indices // corrupted_test_df.shape[1]
cols = random_indices % corrupted_test_df.shape[1]

# 랜덤 이상치 값 선택하여 대체
for i in range(n_outliers):
    # t_outliers에서 랜덤하게 하나 선택
    outlier_value = np.random.choice(t_outliers)
    r, c = rows[i], cols[i]
    corrupted_test_df.iloc[r, c] = outlier_value

# 4. 결측치 추가 - 결측치 비율만큼 랜덤 데이터 선택하고 NaN으로 대체
# 추가할 결측치 개수
n_missing = int(total_points * missing_ratio)

# 이상치와 겹치지 않도록 나머지 데이터 중에서 선택
remaining_indices = np.setdiff1d(np.arange(total_points), random_indices)
missing_indices = np.random.choice(remaining_indices, n_missing, replace=False)

# 결측치 위치 계산
missing_rows = missing_indices // corrupted_test_df.shape[1]
missing_cols = missing_indices % corrupted_test_df.shape[1]

# NaN으로 대체
for i in range(n_missing):
    r, c = missing_rows[i], missing_cols[i]
    corrupted_test_df.iloc[r, c] = np.nan

# 결과 확인
print(f"원본 test_df 크기: {test_df.shape}")
print(f"손상된 test_df 크기: {corrupted_test_df.shape}")
print(f"추가된 이상치 개수: {n_outliers} (비율: {outlier_ratio:.4f})")
print(f"추가된 결측치 개수: {n_missing} (비율: {missing_ratio:.4f})")
print(f"손상된 데이터의 실제 결측치 비율: {corrupted_test_df.isna().sum().sum() / corrupted_test_df.size:.4f}")

In [ ]:
# 1. 원본과 손상된 데이터 비교 (몇 개 컬럼 샘플링)
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('원본 데이터와 손상된 데이터 비교', fontsize=16)

# 랜덤으로 4개 컬럼 선택
sample_cols = np.random.choice(test_df.columns, 4, replace=False)

for i, col in enumerate(sample_cols):
    ax = axes[i//2, i%2]
    
    # 원본 데이터
    ax.plot(test_df.index, test_df[col], 'b-', alpha=0.7, label='원본')
    
    # 손상된 데이터
    ax.plot(corrupted_test_df.index, corrupted_test_df[col], 'r.', alpha=0.7, label='손상됨')
    
    ax.set_title(f'컬럼: {col}')
    ax.legend()
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

# 2. 결측치 히트맵
plt.figure(figsize=(14, 8))
plt.title('결측치 분포 히트맵', fontsize=16)
sns.heatmap(corrupted_test_df.isna(), cmap='viridis', cbar_kws={'label': '결측 여부'})
plt.tight_layout()
plt.show()

# 3. 이상치 시각화
# z-score 계산
def get_zscore(df):
    return (df - df.mean()) / df.std()

z_scores = get_zscore(corrupted_test_df)
is_outlier = (np.abs(z_scores) > 3) & (~corrupted_test_df.isna())

plt.figure(figsize=(14, 8))
plt.title('이상치(|z-score| > 3) 분포 히트맵', fontsize=16)
sns.heatmap(is_outlier, cmap='OrRd', cbar_kws={'label': '이상치 여부'})
plt.tight_layout()
plt.show()

# 4. 데이터 분포 비교 (상자 그림) - 수정된 버전
plt.figure(figsize=(16, 10))
plt.title('원본 vs 손상된 데이터 분포 비교 (상자 그림)', fontsize=16)

# 인덱스 리셋 및 일부 특성만 선택하여 시각화
# sample_cols에서 이미 선택한 4개 컬럼만 사용
orig_sample = test_df[sample_cols].reset_index()
corr_sample = corrupted_test_df[sample_cols].reset_index()

# 데이터 재구성
orig_melt = orig_sample.melt(id_vars='index', var_name='특성', value_name='값')
orig_melt['데이터셋'] = '원본'
corr_melt = corr_sample.melt(id_vars='index', var_name='특성', value_name='값')
corr_melt['데이터셋'] = '손상됨'

# 결합 및 시각화
combined = pd.concat([orig_melt, corr_melt]).reset_index(drop=True)
sns.boxplot(x='특성', y='값', hue='데이터셋', data=combined, showfliers=True)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# 5. 일부 특성에 대한 값 분포 비교 (히스토그램)
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('원본과 손상된 데이터의 분포 비교 (히스토그램)', fontsize=16)

for i, col in enumerate(sample_cols):
    ax = axes[i//2, i%2]
    sns.histplot(test_df[col], color='blue', alpha=0.5, label='원본', ax=ax)
    sns.histplot(corrupted_test_df[col], color='red', alpha=0.5, label='손상됨', ax=ax)
    ax.set_title(f'컬럼: {col}')
    ax.legend()

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
comb_df = pd.concat([training_df, corrupted_test_df], axis=0, copy=True)
print(comb_df.shape)
print(comb_df.index.is_monotonic_increasing)
comb_df